In [13]:
"""
Figma API 접근성 체크 모듈
"""

import os
import requests
import logging
import re
import json
from pathlib import Path
from dotenv import load_dotenv
from typing import Tuple, Optional

# 로깅 설정
logging.basicConfig(level=logging.INFO, format="%(message)s")
logger = logging.getLogger(__name__)


class FigmaAuth:
    def __init__(self):
        self.token = None
        self.file_id = None

    def load_env(self) -> bool:
        """환경 변수 로드"""
        try:
            # .env 파일 찾기
            env_path = Path.cwd().parent / ".env"
            if not env_path.exists():
                logger.error("❌ .env 파일 없음")
                return False

            logger.info(f"📁 .env 파일 위치: {env_path}")

            # 환경 변수 로드
            load_dotenv(env_path)

            # 필수 값 확인
            self.token = os.getenv("FIGMA_ACCESS_TOKEN")
            self.file_id = os.getenv("FIGMA_FILE_ID")

            if not self.token or not self.file_id:
                logger.error("❌ 필수 환경 변수 누락")
                logger.error("- FIGMA_ACCESS_TOKEN")
                logger.error("- FIGMA_FILE_ID")
                return False

            # 토큰 정보 출력 (마지막 4자리만)
            token_preview = (
                f"{self.token[:8]}...{self.token[-4:]}" if self.token else "None"
            )
            logger.info(f"🔑 토큰: {token_preview}")
            logger.info(f"📄 토큰 길이: {len(self.token) if self.token else 0}")

            return True

        except Exception as e:
            logger.error(f"❌ 환경 변수 로드 실패: {e}")
            return False

    def validate_token(self) -> bool:
        """토큰 유효성 검증"""
        if not self.token:
            return False

        if not self.token.startswith("figd_"):
            logger.error("❌ 잘못된 토큰 형식")
            logger.error("Figma 토큰은 'figd_'로 시작해야 함")
            return False

        return True

    def test_api(self) -> bool:
        """API 접근 테스트"""
        try:
            # 1. 토큰 검증
            if not self.validate_token():
                return False

            # 2. API 요청
            url = f"https://api.figma.com/v1/files/{self.file_id}/versions"
            headers = {"X-Figma-Token": self.token}

            logger.info("\n🔍 Figma API 테스트 중...")
            logger.info(f"- File ID: {self.file_id}")

            response = requests.get(url, headers=headers)

            # 3. 응답 확인
            if response.status_code == 200:
                logger.info("✅ API 접근 성공!")
                return True

            # 4. 오류 처리
            error = response.json()
            logger.error(f"❌ API 오류: {error}")

            if response.status_code == 403:
                if "Invalid token" in error.get("err", ""):
                    logger.error("토큰이 유효하지 않음")
                    logger.error("1. Figma 계정 설정 확인")
                    logger.error("2. 새 토큰 발급 (files:read 권한 필요)")
                else:
                    logger.error("권한 부족")
                    logger.error("files:read 권한 필요")

            elif response.status_code == 404:
                logger.error("파일을 찾을 수 없음")
                logger.error("File ID를 확인하세요")

            return False

        except requests.exceptions.RequestException as e:
            logger.error(f"❌ 네트워크 오류: {e}")
            return False

        except Exception as e:
            logger.error(f"❌ 예상치 못한 오류: {e}")
            return False


def check_figma_access():
    """Figma API 접근성 검증"""
    auth = FigmaAuth()
    if auth.load_env():
        return auth.test_api()
    return False


if __name__ == "__main__":
    check_figma_access()


2025-04-04 20:01:18,858 - INFO - 📁 .env 파일 위치: /Users/dyha/Documents/GitHub/figma/cdsMaker/src/utils-mcp/figma-mcp-server/.env
2025-04-04 20:01:18,859 - INFO - 🔑 토큰: figd_cqB..._8or
2025-04-04 20:01:18,859 - INFO - 📄 토큰 길이: 45
2025-04-04 20:01:18,860 - INFO - 
🔍 Figma API 테스트 중...
2025-04-04 20:01:18,860 - INFO - - File ID: 5n51rBzxD4qAvPGdLDlozn
2025-04-04 20:01:19,062 - ERROR - ❌ API 오류: {'status': 403, 'err': 'Invalid token'}
2025-04-04 20:01:19,062 - ERROR - 토큰이 유효하지 않음
2025-04-04 20:01:19,063 - ERROR - 1. Figma 계정 설정 확인
2025-04-04 20:01:19,063 - ERROR - 2. 새 토큰 발급 (files:read 권한 필요)
